In [2]:
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import datetime as dt

In [10]:
# Define the basket's tickers and weights
tickers = ['BHP.AX', 'CSL.AX', 'WDS.AX', 'MQG.AX']
weights = np.array([0.1, 0.35, 0.15, 0.4])

In [16]:
# Extracting closing prices as of Friday, 16 May 2025 from yfinance
data = yf.download(tickers, start='2025-05-16', end='2025-05-17')['Close']

# Extract the closing prices for that date
closing_prices = data.loc['2025-05-16'].values

# Calculate weighted average (basket spot price S0)
S0 = np.dot(weights, closing_prices)

print("Basket spot price S0 as of 16 May 2025:", S0)
print("Closing prices on 16 May 2025:", closing_prices)

[*********************100%***********************]  4 of 4 completed

Basket spot price S0 as of 16 May 2025: 128.4795036315918
Closing prices on 16 May 2025: [ 39.72000122 241.82000732 207.3500061   21.92000008]


In [18]:
def binomial_european_call(S0, K, T, sigma, N):
    """Calculate European call option price using binomial model"""
    # Calculate time step
    dt = T/N
    print(f"dt: {dt}")
    
    # Calculate up and down factors
    u = np.exp(sigma * np.sqrt(dt))
    print(f"u: {u}")
    d = 1/u
    print(f"d: {d}")
    
    # Calculate risk-neutral probability
    p = 1 / (u + 1)
    print(f"p: {p}")
    
    # Initialize stock price tree
    stock_tree = np.zeros((N+1, N+1))
    
    # Populate the stock price tree through time
    for i in range(N+1):
        for j in range(i+1):
            stock_tree[i, j] = S0 * (u ** j) * (d ** (i - j))
    
    # Initialize option value tree
    option_tree = np.zeros((N+1, N+1))
    
    # Calculate call option payoffs at expiration
    for j in range(N+1):
        option_tree[N, j] = max(0, stock_tree[N, j] - K)
    
    # Work backwards discounting and reflecting risk-neutral probabilities to calculate option value at each node
    for i in range(N-1, -1, -1):
        for j in range(i+1):
            option_tree[i, j] = (p * option_tree[i+1, j+1] + (1-p) * option_tree[i+1, j])
    
    return option_tree[0, 0], stock_tree, option_tree

In [21]:
K = 175
valuation_date = dt. datetime(2025, 5, 16)
expiry_date = dt.datetime(2025, 7, 17)
T = (expiry_date - valuation_date).days / 365
sigma = 0.2  # Replace with your basket's volatility estimate
N = 100

price, stock_tree, option_tree = binomial_european_call(S0, K, T, sigma, N)
print(f"European call basket option price: {price:.2f} AUD")
print("Time until expiry (T):", T)

dt: 0.0016986301369863012
u: 1.0082769543153784
d: 0.9917909912748144
p: 0.4979392896239752
European call basket option price: 0.00 AUD
Time until expiry (T): 0.16986301369863013
